# **Anomaly Detection of IoT Devices using Federated Learning**

**Group - 4**

---
***KDD cup (Network communication Dataset)***


In [1]:
import pandas as pd
pd.set_option("display.max_columns", 10)
# KDD CUP 1999 data (https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html)
# Column names are taken from https://kdd.ics.uci.edu/databases/kddcup99/kddcup.names
# There are 41 columns in the data set
colnames = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
            'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
            'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files',
            'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate',
            'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
            'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
            'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
            'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
            'dst_host_srv_rerror_rate']


# Selecting just the first 100K records from the KDD dataset
df = pd.read_csv("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",names=colnames+["threat_type"])[:100000]

df.head()

,duration,protocol_type,service,flag,src_bytes,...,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,threat_type
0,0,tcp,http,SF,181,...,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,...,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,...,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,...,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,...,0.0,0.0,0.0,0.0,normal.


In [2]:
import plotly.graph_objects as go
from collections import Counter

threat_count_dict = Counter(df["threat_type"])
threat_types = list(threat_count_dict.keys())
threat_counts = [threat_count_dict[threat_type] for threat_type in threat_types]
fig = go.Figure([go.Bar(x=threat_types, y=threat_counts,text=threat_counts,textposition='auto')])
fig.show()

In [3]:
print("Total distinct number of threat types : ",len(threat_types))

Total distinct number of threat types :  20


The normality of most communications are normal. Only a few columns are classified as anomalous.

Feature Selection and Scaling done below:

In [4]:
# Considering only the appropriate features as per paper on kdd cup
numerical_colmanes = ['duration', 'src_bytes', 'dst_bytes', 'wrong_fragment', 'urgent', 'hot',
                      'num_failed_logins', 'num_compromised', 'root_shell', 'su_attempted', 'num_root',
                      'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'count',
                      'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
                      'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
                      'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                      'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
                      'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']

numerical_df = df[numerical_colmanes].copy()
# Removing the numerical columns with constant value
numerical_df = numerical_df.loc[:, (numerical_df != numerical_df.iloc[0]).any()]
# lets scale the values for each column from [0,1]
final_df = numerical_df/numerical_df.max()
X = final_df.values
# final dataframe has 33 features
print("Shape of feature matrix : ",X.shape)

Shape of feature matrix :  (100000, 33)


In [5]:
final_df.head()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,...,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,2.610418e-07,0.001057,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
1,0.0,3.446905e-07,0.000094,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
2,0.0,3.389216e-07,0.000259,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
3,0.0,3.158461e-07,0.000259,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
4,0.0,3.129617e-07,0.000394,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0


**using LabelEncoder to encode the threat types in numeric values :**

In [6]:
from sklearn.preprocessing import LabelEncoder

threat_types = df["threat_type"].values
encoder = LabelEncoder()
y = encoder.fit_transform(threat_types)
print("Shape of target vector : ",y.shape)

Shape of target vector :  (100000,)


In [7]:
import numpy as np
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

**Train Test Split**

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.4, random_state=42, stratify=y)
print("Number of records in training data : ", X_train.shape)
print("Number of records in test data : ", X_test.shape)
print("Total distinct number of threat types in training data : ",len(set(y_train)))
print("Total distinct number of threat types in test data : ",len(set(y_test)))

Number of records in training data :  (60000, 33)
Number of records in test data :  (40000, 33)
Total distinct number of threat types in training data :  20
Total distinct number of threat types in test data :  20


In [0]:
#pip uninstall protobuf

In [0]:
#!pip install syft

In [0]:
import torch as th
import syft as sy

# Hook PyTorch ie add extra functionalities to support Federated Learning
hook = sy.TorchHook(th)
# Sets the seed for generating random numbers.
th.manual_seed(1)
# Select CPU computation, in case you want GPU use "cuda" instead
device = th.device("cpu")
# Data will be distributed among these VirtualWorkers.
# Remote training of the model will happen here: gateway1 and gateway2
gateway1 = sy.VirtualWorker(hook, id="gateway1")
gateway2 = sy.VirtualWorker(hook, id="gateway2")

In [11]:
# Number of times we want to iterate over whole training data
BATCH_SIZE = 1000
EPOCHS = 5
LOG_INTERVAL = 5
lr = 0.01

n_feature = X_train.shape[1]
n_class = np.unique(y_train).shape[0]

print("Number of training features : ",n_feature)
print("Number of training classes : ",n_class)

Number of training features :  33
Number of training classes :  20


In [0]:
# Create pytorch tensor from X_train,y_train,X_test,y_test
train_inputs = th.tensor(X_train,dtype=th.float)
train_labels = th.tensor(y_train)
test_inputs = th.tensor(X_test,dtype=th.float)
test_labels = th.tensor(y_test)

# Send the training and test data to the gatways in equal proportion.
# since there are two gateways we are splitting into two : 
train_idx = int(len(train_labels)/2)
test_idx = int(len(test_labels)/2)
gateway1_train_dataset = sy.BaseDataset(train_inputs[:train_idx], train_labels[:train_idx]).send(gateway1)
gateway2_train_dataset = sy.BaseDataset(train_inputs[train_idx:], train_labels[train_idx:]).send(gateway2)
gateway1_test_dataset = sy.BaseDataset(test_inputs[:test_idx], test_labels[:test_idx]).send(gateway1)
gateway2_test_dataset = sy.BaseDataset(test_inputs[test_idx:], test_labels[test_idx:]).send(gateway2)

# Create federated datasets, an extension of Pytorch TensorDataset class
federated_train_dataset = sy.FederatedDataset([gateway1_train_dataset, gateway2_train_dataset])
federated_test_dataset = sy.FederatedDataset([gateway1_test_dataset, gateway2_test_dataset])

# Create federated dataloaders, an extension of Pytorch DataLoader class
federated_train_loader = sy.FederatedDataLoader(federated_train_dataset, shuffle=True, batch_size=BATCH_SIZE)
federated_test_loader = sy.FederatedDataLoader(federated_test_dataset, shuffle=False, batch_size=BATCH_SIZE)

**Initialising Neural Network**

In [0]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        """
        input_dim: number of input features.
        output_dim: number of labels.
        """
        super(Net, self).__init__()
        self.linear = th.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        outputs = self.linear(x)
        return outputs

**Training using Neural Network**

In [0]:
import torch.nn.functional as F

def train(model, device, federated_train_loader, optimizer, epoch):
    model.train()
    # Iterate through each gateway's dataset
    for idx, (data, target) in enumerate(federated_train_loader):
        batch_idx = idx+1
        # Send the model to the right gateway
        model.send(data.location)
        # Move the data and target labels to the device (cpu/gpu) for computation
        data, target = data.to(device), target.to(device)
        # Clear previous gradients (if they exist)
        optimizer.zero_grad()
        # Make a prediction
        output = model(data)
        # Calculate the cross entropy loss [We are doing classification]
        loss = F.cross_entropy(output, target)
        # Calculate the gradients
        loss.backward()
        # Update the model weights
        optimizer.step()
        # Get the model back from the gateway
        model.get()
        if batch_idx==len(federated_train_loader) or (batch_idx!=0 and batch_idx % LOG_INTERVAL == 0):
            # get the loss back
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * BATCH_SIZE, len(federated_train_loader) * BATCH_SIZE,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [0]:
def test(model, device, federated_test_loader):
    model.eval()
    correct = 0
    with th.no_grad():
        for batch_idx, (data, target) in enumerate(federated_test_loader):
            # Send the model to the right gateway
            model.send(data.location)
            # Move the data and target labels to the device (cpu/gpu) for computation
            data, target = data.to(device), target.to(device)
            # Make a prediction
            output = model(data)
            # Get the model back from the gateway
            model.get()
            # Calculate the cross entropy loss
            loss = F.cross_entropy(output, target)
            # Get the index of the max log-probability 
            pred = output.argmax(1, keepdim=True)
            # Get the number of instances correctly predicted
            correct += pred.eq(target.view_as(pred)).sum().get()
                
    # get the loss back
    loss = loss.get()
    print('Test set: Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss.item(), correct, len(federated_test_loader.federated_dataset),
        100. * correct / len(federated_test_loader.federated_dataset)))

In [16]:
%%time
import torch.optim as optim

# Initialize the model
model = Net(n_feature,n_class)

#Initialize the SGD optimizer
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(1, EPOCHS + 1):
    # Train on the training data in a federated way
    train(model, device, federated_train_loader, optimizer, epoch)
    # Check the test accuracy on unseen test data in a federated way
    test(model, device, federated_test_loader)

Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.996903
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.905353
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.822427
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.747468
Train Epoch: 1 [25000/60000 (42%)]	Loss: 2.683699
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.608650
Train Epoch: 1 [35000/60000 (58%)]	Loss: 2.537718
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.462108
Train Epoch: 1 [45000/60000 (75%)]	Loss: 2.411441
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.354580
Train Epoch: 1 [55000/60000 (92%)]	Loss: 2.288137
Train Epoch: 1 [60000/60000 (100%)]	Loss: 2.205424
Test set: Loss: 2.2076, Accuracy: 22493/40000 (56%)

Train Epoch: 2 [5000/60000 (8%)]	Loss: 2.179445
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.103422
Train Epoch: 2 [15000/60000 (25%)]	Loss: 2.043215
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.986939
Train Epoch: 2 [25000/60000 (42%)]	Loss: 1.953733
Train Epoch: 2 [30000/60000 (50%)]	Loss: 1.873909
Train Epoch: 2 [35000/60000 (58%)]	Loss: 1.835567


# **Saving the Model and Loading it for Prediction**

In [17]:
from google.colab import files
# Save the model
th.save(model.state_dict(), "network-threat-kdd-model.pt")
#Download Model
files.download('network-threat-kdd-model.pt')
# Reload the model in a new model object
model_new = Net(n_feature,n_class)
model_new.load_state_dict(th.load("network-threat-kdd-model.pt"))
model_new.eval()

Net(
  (linear): Linear(in_features=33, out_features=20, bias=True)
)

In [18]:
# Take the 122th record from the test data
idx = 122
data = test_inputs[idx]
pred = model_new(data)
pred_label = int(pred.argmax().data.cpu().numpy())
pred_threat = encoder.inverse_transform([pred_label])[0]
print("Predicted threat type : ", pred_threat)
actual_label = int(test_labels[idx].data.cpu().numpy())
actual_threat = encoder.inverse_transform([actual_label])[0]
print("Actual threat type : ", actual_threat)

Predicted threat type :  neptune.
Actual threat type :  neptune.


In [19]:
# Take the 159th record from the test data
idx = 100
data = test_inputs[idx]
pred = model_new(data)
pred_label = int(pred.argmax().data.cpu().numpy())
pred_threat = encoder.inverse_transform([pred_label])[0]
print("Predicted threat type : ", pred_threat)
actual_label = int(test_labels[idx].data.cpu().numpy())
actual_threat = encoder.inverse_transform([actual_label])[0]
print("Actual threat type : ", actual_threat)

Predicted threat type :  normal.
Actual threat type :  normal.
